<a href="https://colab.research.google.com/github/Shaffizy/Sentimental-Analysist/blob/main/Shaffi's_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing the Dependencies**
In this code, I imported the following dependencies: Pandas, Scikit-learn, Tensorflow.
Pandas for to visualize the data in this this programming environment
Scikit-Learn to divide the data to test and train data.
Tensorflow for the training of the date when it is split to train and test.We would use the Dense, Embedding and LSTM.

In [5]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

**Data Collection-kaggle API**
This function is used to load the file "kaggle.json" which contains my kaggle information (name, password) to enable the download of a particular kaggle file called "imdb-dataset-of-50k-movie-reviews".

In [6]:
def Data_Collection():
    kaggle_dictionary = json.load(open("kaggle.json"))
    os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
    os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]
Data_Collection()

In [7]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 97% 25.0M/25.7M [00:01<00:00, 26.6MB/s]
100% 25.7M/25.7M [00:01<00:00, 16.0MB/s]


**Data Extraction**
This function is used to unzip the "imdb-dataset-of-50k-movie-reviews" so it can be visualized by pandas.


In [8]:
def Data_Extraction():
  with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
        zip_ref.extractall()
Data_Extraction()


**Data Processing**
In this function I used pandas to read through the data.Then I replaced the "positive" and "negative" value in the "sentiment" column of the *data* that is the variable of the result of the it's visualization by pandas to integer so the machine can read and understand. After that I defined the *train* and *test* data by spliting them with scikit-learn. Tokenization is now carried out to replace the word with corresponding numbers so the machine can easily work with it. While X(train, test) and Y(train, test) are then initialized with the former being the data of values of "review" and the latter beign the data of values of "sentiment".

In [15]:
def Data_Processing():
    data = pd.read_csv("/content/IMDB Dataset.csv")
    data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)
    train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
    print(train_data.shape)
    print(test_data.shape)

    return train_data , test_data

In [19]:
# Initialize the values of train and test data
train_data , test_data = Data_Processing()

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])

X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)
print(X_train)
print(X_test)

Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]
print(Y_train)
print(Y_test)

(40000, 2)
(10000, 2)
[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]
[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]
39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64
33553    1
9427     1
199      0
12447    1
39489    0
        ..
28567    0
25079    1
18707    1
15200    0
5857     1
Name: sentiment, Length: 10000, dtype: int64


**LSTM - Long Short-Term Memory**
I used this function to build the model with Embedding, LSTM, Dense provided by tensorflow and then I compile the model together so everything works simultaneously in concurrent order and I train the model the finally carry out model evaluation which shows the accuracy of the machine  

In [17]:
def Trainning():
    # build the model

    model = Sequential()
    model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
    model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(1, activation="sigmoid"))
    model.summary()

    return model


In [18]:
# compile the model
model = Trainning()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# training the Model
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

# model Evaluation
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 242s 480ms/step - accuracy: 0.7300 - loss: 0.5215 - val_accuracy: 0.7596 - val_loss: 0.4861
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 244s 445ms/step - accuracy: 0.8334 - loss: 0.3829 - val_accuracy: 0.8026 - val_loss: 0.4219
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 225s 450ms/step - accuracy: 0.8638 - loss: 0.3340 - val_accuracy: 0.8655 - val_loss: 0.3295
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 268s 464ms/step - accuracy: 0.8899 - loss: 0.2752 - val_accuracy: 0.8676 - val_loss: 0.3352
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 222s 443ms/step - accuracy: 0.9054 - loss: 0.2419 - val_accuracy: 0.8745 - val_loss: 0.3194
313/313 ━━━━━━━━━━━━━━━━━━━━ 32s 103ms/step - accuracy: 0.8789 - loss: 0.3037
Test Loss: 0.3009601831436157
Test Accuracy: 0.8820000290870667


**Building a Predictive System** Testing and verification of what the machine has learnt would happen here. I would test various test case with original reviews.


In [20]:
def predict_sentiment(review):

  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"

  return sentiment

In [21]:
# example usage
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step
The sentiment of the review is: positive
